In [ ]:
import pandas as pd

data = pd.read_excel('Data_Mining_Phish+Malware_Combined_Class_Imbalance.xlsx')

In [ ]:
data['Label'].replace({'INVALID': 'Safe', 'malware_download': 'Malware', 'VALID PHISH': 'Phish'}, inplace=True)

In [ ]:
data_cl = data.drop_duplicates()
data_cl

,URL,Label
0,http://182.112.191.199:49467/bin.sh,Malware
1,http://182.116.123.56:58251/i,Malware
2,http://182.116.123.56:58251/bin.sh,Malware
3,http://49.73.48.99:49444/.i,Malware
4,http://112.248.253.129:44460/bin.sh,Malware
...,...,...
80146,http://raceforthekids.hk/,Safe
80147,http://www.tested.com/tech/robots/532970-teste...,Safe
80148,http://odtapuch.com/Mail/CountLink.aspx?SID=54...,Safe
80149,http://pornkip.com/2016/03/20/anya-ivy-diamond...,Safe


In [ ]:
counts = data_cl.groupby('Label').size()
print(counts)

Label
Malware    28424
Phish      25287
Safe       23592
dtype: int64


##Feature Engineering

#URL Length

In [ ]:
def url_length(url):
    return len(url)


# Apply the functions to the URL column to create new features
data_cl['URL_length'] = data_cl['URL'].apply(url_length)
data_cl.head()


<ipython-input-8-a182c045669b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['URL_length'] = data_cl['URL'].apply(url_length)


,URL,Label,URL_length
0,http://182.112.191.199:49467/bin.sh,Malware,35
1,http://182.116.123.56:58251/i,Malware,29
2,http://182.116.123.56:58251/bin.sh,Malware,34
3,http://49.73.48.99:49444/.i,Malware,27
4,http://112.248.253.129:44460/bin.sh,Malware,35


#HostName Length

In [ ]:
import urllib.parse

def hostname_len(url):
    parsed_url = urllib.parse.urlparse(url)
    hostname = parsed_url.hostname
    if hostname is not None:
        hostname_length = len(hostname)
    else:
        hostname_length = 0
    return hostname_length

data_cl['URL_HostName_length'] = data_cl['URL'].apply(hostname_len)
data_cl.head()

<ipython-input-9-6edb4ade00f7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['URL_HostName_length'] = data_cl['URL'].apply(hostname_len)


,URL,Label,URL_length,URL_HostName_length
0,http://182.112.191.199:49467/bin.sh,Malware,35,15
1,http://182.116.123.56:58251/i,Malware,29,14
2,http://182.116.123.56:58251/bin.sh,Malware,34,14
3,http://49.73.48.99:49444/.i,Malware,27,11
4,http://112.248.253.129:44460/bin.sh,Malware,35,15


#Length of top-level domain

In [ ]:
!pip install tld
import tld
from tld import get_tld
def extract_tld(url):
    td = get_tld(url,fail_silently=True)
    if td is not None:
        return td
    else:
        return ''

data_cl['Top_Level_Domain'] = data_cl['URL'].apply(extract_tld)

data_cl['Top_Level_Domain_Length'] = data_cl['Top_Level_Domain'].apply(len)

# Print the updated dataframe with the new column
data_cl.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 12.0 MB/s eta 0:00:00


<ipython-input-10-db04f823f63d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Top_Level_Domain'] = data_cl['URL'].apply(extract_tld)
<ipython-input-10-db04f823f63d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Top_Level_Domain_Length'] = data_cl['Top_Level_Domain'].apply(len)


,URL,Label,URL_length,URL_HostName_length,Top_Level_Domain,Top_Level_Domain_Length
0,http://182.112.191.199:49467/bin.sh,Malware,35,15,,0
1,http://182.116.123.56:58251/i,Malware,29,14,,0
2,http://182.116.123.56:58251/bin.sh,Malware,34,14,,0
3,http://49.73.48.99:49444/.i,Malware,27,11,,0
4,http://112.248.253.129:44460/bin.sh,Malware,35,15,,0


##Dot Count (Multiple Domains)

In [ ]:
def is_suspicious(url):
    dot_count = url.count('.')
    if dot_count > 3:
      return 1
    else:
      return 0

data_cl['dot_count'] = data_cl['URL'].apply(is_suspicious)
data_cl


<ipython-input-11-8068aa762942>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['dot_count'] = data_cl['URL'].apply(is_suspicious)


,URL,Label,URL_length,URL_HostName_length,Top_Level_Domain,Top_Level_Domain_Length,dot_count
0,http://182.112.191.199:49467/bin.sh,Malware,35,15,,0,1
1,http://182.116.123.56:58251/i,Malware,29,14,,0,0
2,http://182.116.123.56:58251/bin.sh,Malware,34,14,,0,1
3,http://49.73.48.99:49444/.i,Malware,27,11,,0,1
4,http://112.248.253.129:44460/bin.sh,Malware,35,15,,0,1
...,...,...,...,...,...,...,...
80146,http://raceforthekids.hk/,Safe,25,17,hk,2,0
80147,http://www.tested.com/tech/robots/532970-teste...,Safe,73,14,com,3,1
80148,http://odtapuch.com/Mail/CountLink.aspx?SID=54...,Safe,69,12,com,3,1
80149,http://pornkip.com/2016/03/20/anya-ivy-diamond...,Safe,73,11,com,3,1


##Count-www

In [ ]:
def count_www(url):
    if url.count('www') == 1:
        return 0
    else:
        return 1

data_cl['Count_WWW'] = data_cl['URL'].apply(count_www)

<ipython-input-12-afe49eddb06c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Count_WWW'] = data_cl['URL'].apply(count_www)


#Presence of @

In [ ]:
def presence_attherate(url):
     
    if url.count('@') > 0:
        return 1
    else:
        return 0
data_cl['@_Presence'] = data_cl['URL'].apply(presence_attherate)

<ipython-input-13-932edc30ccc8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['@_Presence'] = data_cl['URL'].apply(presence_attherate)


#Multiple Directories

In [ ]:
def count_dir(url):
    path = urllib.parse.urlparse(url).path
    return path.count('/')

data_cl['Multiple_Dir_Count'] = data_cl['URL'].apply(count_dir)

<ipython-input-14-e37678396c04>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Multiple_Dir_Count'] = data_cl['URL'].apply(count_dir)


#Presence_of_Embedded_Domians

In [ ]:
def presence_of_embed_domain(url):
    return url.count('//') - 1
data_cl['Presence_of_Embedded_Domains'] = data_cl['URL'].apply(presence_of_embed_domain)

<ipython-input-15-b027ba8f6834>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Presence_of_Embedded_Domains'] = data_cl['URL'].apply(presence_of_embed_domain)


##Suspecious_Words

In [ ]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

import re

suspicious_words =  ['account', 'bank', 'login', 'secure', 'confirm', 'verify', 'update', 'password',
                    'credit', 'card', 'billing', 'paypal', 'amazon', 'apple', 'netflix', 'google', 'shop'
                    'microsoft', 'yahoo', 'linkedin', 'facebook', 'instagram', 'twitter', 'whatsapp',
                    'irs', 'cra', 'revenue', 'census', 'gov', 'police', 'detective', 'investigator',
                    'lawyer', 'attorney', 'legal', 'court', 'judge', 'bailiff', 'jury', 'warrant',
                    'virus', 'malware', 'trojan', 'spyware', 'phishing', 'spam', 'scam', 'fraud',
                    'fake', 'spoof', 'hoax', 'deception', 'trick', 'beware', 'warning', 'alert',
                    'urgent', 'emergency', 'critical', 'important', 'update', 'notification',
                    'accountupdate', 'banking', 'billing', 'security', 'verify', 'confirm',
                    'logon', 'ssl', 'session', 'bankofamerica', 'wellsfargo', 'chase', 'citibank',
                    'paypal', 'amazon', 'apple', 'netflix', 'google', 'microsoft', 'yahoo', 'linkedin',
                    'facebook', 'instagram', 'twitter', 'whatsapp', 'irs', 'cra', 'revenue', 'census',
                    'gov', 'police', 'detective', 'investigator', 'lawyer', 'attorney', 'legal',
                    'court', 'judge', 'bailiff', 'jury', 'warrant', 'virus', 'malware', 'trojan',
                    'spyware', 'phishing', 'spam', 'scam', 'fraud', 'fake', 'spoof', 'hoax',
                    'deception', 'trick', 'beware', 'warning', 'alert', 'urgent', 'emergency',
                    'critical', 'important', 'update', 'notification', 'accountupdate', 'banking',
                    'billing', 'security', 'verify', 'confirm', 'logon', 'ssl', 'session', 'click',
                    'download', 'install', 'upgrade', 'get', 'free', 'discount', 'offer', 'sale',
                    'win', 'prize', 'winner', 'congratulations', 'gift', 'claim', 'limited',
                    'time', 'hurry', 'now', 'act', 'fast', 'exclusive', 'special', 'deal', 'buy',
                    'order', 'purchase', 'subscribe', 'unsubscribe', 'newsletter', 'mailing',
                    'list', 'survey', 'opinion', 'research', 'study', 'test', 'trial', 'request',
                    'help', 'support', 'customer', 'service', 'complaint', 'refund', 'return',
                    'cancel', 'delete', 'remove', 'profile', 'privacy', 'terms', 'conditions',
                    'policy', 'legal', 'copyright', 'trademark',
                    'pharmacy', 'meds', 'drug', 'pills', 'prescription', 'rx', 'doctor', 'medical',
                    'health', 'insurance', 'wellness', 'weight', 'loss']

# create a function to lemmatize a word using WordNetLemmatizer from nltk
def lemmatize(word):
    lemma = wordnet.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

# apply lemmatization to each word in the suspicious_words list
suspicious_words_lemmatized = [lemmatize(word) for word in suspicious_words]

# join the lemmatized words with the '|' character to create a regex pattern
pattern = '|'.join(suspicious_words_lemmatized)

# create a function to check for the presence of the pattern in a URL
def check_url(url):
    check = re.search(pattern, url)
    if check:
        return 1
    else:
        return 0

# apply the check_url function to the 'URL' column of the data_cl dataframe
data_cl['suspicious_words'] = data_cl['URL'].apply(check_url)

[nltk_data] Downloading package wordnet to /root/nltk_data...
<ipython-input-16-5bda9b04e823>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['suspicious_words'] = data_cl['URL'].apply(check_url)


In [ ]:
data_cl['suspicious_words'].sum()

14354

#Presence_of_https

In [ ]:
import re
import re

def has_https(url):
    if re.search('https', url):
        return 1
    else:
        return 0

data_cl['Presence_of_https'] = data_cl['URL'].apply(has_https)

<ipython-input-18-e7200fe9ad9c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Presence_of_https'] = data_cl['URL'].apply(has_https)


In [ ]:
data_cl

,URL,Label,URL_length,URL_HostName_length,Top_Level_Domain,Top_Level_Domain_Length,dot_count,Count_WWW,@_Presence,Multiple_Dir_Count,Presence_of_Embedded_Domains,suspicious_words,Presence_of_https
0,http://182.112.191.199:49467/bin.sh,Malware,35,15,,0,1,1,0,1,0,0,0
1,http://182.116.123.56:58251/i,Malware,29,14,,0,0,1,0,1,0,0,0
2,http://182.116.123.56:58251/bin.sh,Malware,34,14,,0,1,1,0,1,0,0,0
3,http://49.73.48.99:49444/.i,Malware,27,11,,0,1,1,0,1,0,0,0
4,http://112.248.253.129:44460/bin.sh,Malware,35,15,,0,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80146,http://raceforthekids.hk/,Safe,25,17,hk,2,0,1,0,1,0,0,0
80147,http://www.tested.com/tech/robots/532970-teste...,Safe,73,14,com,3,1,0,0,3,0,1,0
80148,http://odtapuch.com/Mail/CountLink.aspx?SID=54...,Safe,69,12,com,3,1,1,0,2,0,0,0
80149,http://pornkip.com/2016/03/20/anya-ivy-diamond...,Safe,73,11,com,3,1,1,0,4,0,0,0


#Count_of_HTTP

In [ ]:
def count_http(url):
    pattern = r'http(?!s)'
    return len(re.findall(pattern, url))

data_cl['Count_of_http'] = data_cl['URL'].apply(count_http)

<ipython-input-20-7f985b4a4403>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Count_of_http'] = data_cl['URL'].apply(count_http)


In [ ]:
data_cl['Count_of_http'].sum()

39993

#Digit_Count

In [ ]:
def count_digits_in_url(url):
    digits_count = len(re.findall(r'\d', url))
    return digits_count

data_cl['Digit_Count'] = data_cl['URL'].apply(count_digits_in_url)

<ipython-input-22-afc403b54ada>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Digit_Count'] = data_cl['URL'].apply(count_digits_in_url)


In [ ]:
data_cl['Digit_Count']

0        17
1        16
2        16
3        13
4        17
         ..
80146     0
80147     6
80148    17
80149     8
80150     1
Name: Digit_Count, Length: 77303, dtype: int64

# Count of ?

In [ ]:
def count_question_marks(url):
    return url.count('?')
data_cl['Count_of_question_marks'] = data_cl['URL'].apply(count_question_marks)

<ipython-input-24-598c9bf09ce1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Count_of_question_marks'] = data_cl['URL'].apply(count_question_marks)


In [ ]:
data_cl['Count_of_question_marks']

0        0
1        0
2        0
3        0
4        0
        ..
80146    0
80147    0
80148    1
80149    0
80150    0
Name: Count_of_question_marks, Length: 77303, dtype: int64

#Count of =

In [ ]:
def count_equals(url):
    return url.count('=')
data_cl['Count_of_equals'] = data_cl['URL'].apply(count_equals)

<ipython-input-26-0168fbef8305>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['Count_of_equals'] = data_cl['URL'].apply(count_equals)


In [ ]:
data_cl

,URL,Label,URL_length,URL_HostName_length,Top_Level_Domain,Top_Level_Domain_Length,dot_count,Count_WWW,@_Presence,Multiple_Dir_Count,Presence_of_Embedded_Domains,suspicious_words,Presence_of_https,Count_of_http,Digit_Count,Count_of_question_marks,Count_of_equals
0,http://182.112.191.199:49467/bin.sh,Malware,35,15,,0,1,1,0,1,0,0,0,1,17,0,0
1,http://182.116.123.56:58251/i,Malware,29,14,,0,0,1,0,1,0,0,0,1,16,0,0
2,http://182.116.123.56:58251/bin.sh,Malware,34,14,,0,1,1,0,1,0,0,0,1,16,0,0
3,http://49.73.48.99:49444/.i,Malware,27,11,,0,1,1,0,1,0,0,0,1,13,0,0
4,http://112.248.253.129:44460/bin.sh,Malware,35,15,,0,1,1,0,1,0,0,0,1,17,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80146,http://raceforthekids.hk/,Safe,25,17,hk,2,0,1,0,1,0,0,0,1,0,0,0
80147,http://www.tested.com/tech/robots/532970-teste...,Safe,73,14,com,3,1,0,0,3,0,1,0,1,6,0,0
80148,http://odtapuch.com/Mail/CountLink.aspx?SID=54...,Safe,69,12,com,3,1,1,0,2,0,0,0,1,17,1,2
80149,http://pornkip.com/2016/03/20/anya-ivy-diamond...,Safe,73,11,com,3,1,1,0,4,0,0,0,1,8,0,0


# URL shortening

In [ ]:
shortening_services = [
    '1link.io', '1url.com', '3.ly', '4sq.com', '7.ly', 'adf.ly', 'aka.gr', 'amzn.to', 'bc.vc', 'bit.do',
    'bit.ly', 'bitly.com', 'bl.ink', 'br4.in', 'buff.ly', 'canurl.com', 'chilp.it', 'clck.ru', 'cli.gs',
    'cliccami.info', 'clickmeter.com', 'clicky.me', 'tiny.cc', 'tr.im', 'ctt.ec', 'cutt.ly', 'db.tt', 'deck.ly',
    'dft.ba', 'dlvr.it', 'doiop.com', 'dwarfurl.com', 'dy.fi', 'eepurl.com', 'fb.me', 'ff.im', 'fur.ly',
    'fuseurl.com', 'fw.to', 'g.co', 'g.co.uk', 'gag.gl', 'gd.is', 'gg.gg', 'git.io', 'gl-amour.ru',
    'glamur.cf', 'goo.gl', 'good.ly', 'grabify.link', 'gr8.com', 'gri.ms', 'gur.ly', 'hex.io', 'hrefshare.com',
    'hsbo.me', 'ht.ly', 'hub.me', 'hubsi.me', 'huff.to', 'hulu.com', 'hurl.me', 'icanhaz.com', 'idek.net',
    'igg.me', 'igg.me.uk', 'is.gd', 'its.my', 'ix.lt', 'j.mp', 'jigsy.com', 'jrnl.ie', 'karmacracy.com',
    'kck.st', 'kl.am', 'klck.me', 'korta.nu', 'krunchd.com', 'lkn.li', 'ln-s.net', 'lru.jp', 'lt.tl',
    'lurl.no', 'mcaf.ee', 'me2.do', 'meme.st', 'mgnet.me', 'migre.me', 'miniurl.com', 'minurl.fr',
    'mke.me', 'mobypicture.com', 'mod.my', 'moourl.com', 'mrte.ch', 'myloc.me', 'myurl.in', 'myurl.to',
    'nbc.co', 'nblo.gs', 'ne1.net', 'net.ms', 'net46.net', 'nxy.in', 'nyti.ms', 'o-x.fr', 'oc1.us',
    'om.ly', 'on.fb.me', 'on.mktw.net', 'onforb.es', 'orz.hm', 'ow.ly', 'pea.to', 'ping.fm', 'pli.gs',
    'pnt.me', 'politi.co', 'post.ly', 'pp.gg', 'ppt.cc', 'ptiturl.com', 'pub.vitrue.com', 'qlnk.net',
    'q.gs', 'qklnk.co', 'qps.ru', 'qu.tc', 'qub.me', 'qy.fi', 'r.ebay.com', 'rb6.me', 'read.bi',
    'readthis.ca', 'redir.ec', 'redir.fr', 'redirects.ca', 'retwt.me', 'ri.ms', 'rickroll.it', 'riz.cz',
    'rurl.org', 'rww.tw', 's4c.in', 's7y.us', 'saf.li']

pattern = '|'.join(shortening_services)

# create a function to check for the presence of the pattern in a URL
def shortening_services_url(url):
    check = re.search(pattern, url)
    if check:
        return 1
    else:
        return 0

# apply the check_url function to the 'URL' column of the data_cl dataframe
data_cl['shortening_services'] = data_cl['URL'].apply(shortening_services_url)

<ipython-input-28-ec4446210351>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl['shortening_services'] = data_cl['URL'].apply(shortening_services_url)


In [ ]:
data_cl.drop('Top_Level_Domain', axis=1,inplace=True)

<ipython-input-29-686841b42835>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cl.drop('Top_Level_Domain', axis=1,inplace=True)


In [ ]:
data_cl.to_csv("featured_data.csv")